<a href="https://colab.research.google.com/github/ursulacotrina/Project01/blob/main/scrapping_parlamento_periodico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01 Install dependencies

In [ ]:
!pip install --upgrade -q pygsheets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=beffd6952d8309d4a615ab0ab8925731f77720b525091b0a1302c1e079a10bf8
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InReleas

In [ ]:
# Permite ejecutar comandos de shell directamente en tu entorno de Colab
%%shell

# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium

# Install xvfb
apt install -y xvfb

# Install Selenium-Profiles
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.6

# pip install https://github.com/kaliiiiiiiiii/Selenium-Profiles/archive/refs/heads/dev.zip # dev-branch

# install python packages
pip install google-colab-shell
pip install webdriver-manager
pip install Pyvirtualdisplay

Executing: /tmp/apt-key-gpghome.JxsaYMVG73/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.gX7k7D9A7B/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.LBWABlBco0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

## 02 Install Packages

In [ ]:
import google.auth
from google.colab import auth
auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [ ]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.common.by import By

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
import warnings
import datetime
from datetime import datetime
import time

## 03 Functions

In [ ]:
def open_gsheets(link, sheet_index):
  sh1 = gc.open_by_url(link)
  wks = sh1[sheet_index]
  data_get = pd.DataFrame(wks.get_all_records())
  return data_get

## 04 Import data

### 04 01 scrapkeys

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1FQeZHvmFS5SAIMgGkpP7KNW7CxWu2cP95SkjZqTCh34/edit#gid=0'
sheet_index = 0
variables1 = open_gsheets(link, sheet_index)
variables1

,var,path,featurestable,code,dataappend
0,titulo,/html/body/app-root/app-publico/div[3]/app-exp...,features,"titulo=driver.find_element(By.XPATH,'/html/bod...","'titulo': titulo if status else None,"
1,periodo,/html/body/app-root/app-publico/div[3]/app-exp...,features,"periodo=driver.find_element(By.XPATH,'/html/bo...","'periodo': periodo if status else None,"
2,legisla,/html/body/app-root/app-publico/div[3]/app-exp...,features,"legisla=driver.find_element(By.XPATH,'/html/bo...","'legisla': legisla if status else None,"
3,fechapresenta,/html/body/app-root/app-publico/div[3]/app-exp...,features,"fechapresenta=driver.find_element(By.XPATH,'/h...",'fechapresenta': fechapresenta if status else ...
4,proponente,/html/body/app-root/app-publico/div[3]/app-exp...,features,"proponente=driver.find_element(By.XPATH,'/html...","'proponente': proponente if status else None,"
5,sumilla,/html/body/app-root/app-publico/div[3]/app-exp...,features,"sumilla=driver.find_element(By.XPATH,'/html/bo...","'sumilla': sumilla if status else None,"
6,observaciones,/html/body/app-root/app-publico/div[3]/app-exp...,features,"observaciones=driver.find_element(By.XPATH,'/h...",'observaciones': observaciones if status else ...
7,autor,/html/body/app-root/app-publico/div[3]/app-exp...,features,"autor=driver.find_element(By.XPATH,'/html/body...","'autor': autor if status else None,"
8,coautor,/html/body/app-root/app-publico/div[3]/app-exp...,features,"coautor=driver.find_element(By.XPATH,'/html/bo...","'coautor': coautor if status else None,"
9,adherentes,/html/body/app-root/app-publico/div[3]/app-exp...,features,"adherentes=driver.find_element(By.XPATH,'/html...","'adherentes': adherentes if status else None,"


### 04 02 Acciones y evaluación

In [ ]:
link = 'https://docs.google.com/spreadsheets/d/1unHBIVHSB_oqrtDQb7bNunzPiC_VT_GDXjzRFyTTyXw/edit#gid=0'
sheet_index = 0
variables2 = open_gsheets(link, sheet_index)
variables2

,Numero,Nombre Corto,Nombre grupo,enlace
0,2871,Interplaza,Comisiones Bancarias,https://wb2server.congreso.gob.pe/spley-portal...
1,3739,"Interplaza, transferencia y pago con otra tarjeta",Comisiones Bancarias,https://wb2server.congreso.gob.pe/spley-portal...
2,3262,Pago con cualquier tarjeta (débito o crédito),Comisiones Bancarias,https://wb2server.congreso.gob.pe/spley-portal...
3,3428,Comisión por transferencia,Comisiones Bancarias,https://wb2server.congreso.gob.pe/spley-portal...
4,4359,Comisión por depósito y retiro de dinero en ve...,Comisiones Bancarias,https://wb2server.congreso.gob.pe/spley-portal...
5,4360,Bloqueo de banca móvil,Comisiones Bancarias,https://wb2server.congreso.gob.pe/spley-portal...
6,4525,Proyecto de Ley 4525 y 4704: Reprogramación de...,Reprogramación de Deudas por Lluvias,https://wb2server.congreso.gob.pe/spley-portal...
7,4704,Proyecto de Ley 4525 y 4704: Reprogramación de...,Reprogramación de Deudas por Lluvias,https://wb2server.congreso.gob.pe/spley-portal...
8,4939,Compensación en cuentas bancarias,Compensación de Cuentas,https://wb2server.congreso.gob.pe/spley-portal...


In [ ]:
numbers = variables2['Numero'].tolist()
numbers

[2871, 3739, 3262, 3428, 4359, 4360, 4525, 4704, 4939]

## 05 Scrapping

In [ ]:
var = variables1['var'][variables1['featurestable']=='features'].tolist()
path = variables1['path'][variables1['featurestable']=='features']
path14 = variables1['path'][13]

estadoscrap = []
autorlink = []
fecha = []
enlace = []
vars=[]
vars = var + ['estadoscrap', 'autorlink', 'fecha', 'enlace']

# Convierte a una lista vacía para cada campo de la lista var
for i in var:
  globals()[str(i)] = []

In [ ]:
# Verificar que se tiene una lista vacía para cada campo
for i in vars:
  print(i, globals()[str(i)])

titulo []
periodo []
legisla []
fechapresenta []
proponente []
sumilla []
observaciones []
autor []
coautor []
adherentes []
grupoparla []
comisiones []
ultimoestado []
estadoscrap []
autorlink []
fecha []
enlace []


In [ ]:
#parameters

dd = pd.DataFrame()
ddt = pd.DataFrame()


#title Start actual driver
from selenium_profiles.webdriver import Chrome
from selenium_profiles.profiles import profiles
from selenium.webdriver.common.by import By  # locate elements
from selenium_profiles.utils.colab_utils import display, showscreen, show_html # virtual display
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime

hora_inicio = datetime.now()

chromedriver_path = ChromeDriverManager(version="90.0.4430.24").install()

profile = profiles.Windows() # or .Android
profile["cdp"]["cores"] = None # Chrome 90 doesn't allow emulating cores :(driver = mydriver.start(profile, uc_driver=False, executable_path=chromedriver_path)

mydriver = Chrome(profile, executable_path=chromedriver_path)

display = display()
display.start_display()
driver = mydriver.start()

#Scrap law projects

for e in numbers:
  #clean
  estadoscrap = []
  autorlink = []
  fecha = []
  enlace = []
  for i in var:
    globals()[str(i)] = []


  print(e)
  hoy = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
  link = 'https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/'+str(e)

  fecha.append(hoy)
  enlace.append(link)
  try:
    driver.get(link)  # test client hints
    time.sleep(2) # Tiempo de espera de 2s para ejecutar lo siguiente
    try:
      try:
        driver.find_element(By.XPATH,'//*[@id="p-fieldset-0-content"]/div/div[1]/div[9]/ul/li[3]/a[2]').click()
      except:
        driver.find_element(By.XPATH,'//*[@id="p-fieldset-1-content"]/div/div[1]/div[9]/ul/li[3]/a[2]').click()  #exp 2412
    except NoSuchElementException:
      print('not button')

    try:
      try:
        autorlin = driver.find_element(By.XPATH,'//*[@id="p-fieldset-0-content"]/div/div[1]/div[8]/ul/li/a').get_attribute('href')
      except StaleElementReferenceException:
        autorlin = ''

      status='no'
      autorlink.append(autorlin)

      for i,j in zip(var,path):
        #fill data into list
        try:
          globals()[str(i)].append(driver.find_element(By.XPATH,j).text)
        except NoSuchElementException:
          globals()[str(i)].append('')
      status = 'yes'
      estadoscrap.append(status)
      print('ok')
    except NoSuchElementException:
      pass
    try:
      table_element = driver.find_element(By.XPATH,path14)
      table_html = table_element.get_attribute('innerHTML')
      table_soup = BeautifulSoup(table_html, 'html.parser')
      table = table_soup.table
      dft = pd.read_html(str(table))[0]
      dft['enlace'] = link
    except NoSuchElementException:
      dft = pd.DataFrame()
  except NoSuchElementException:
    for i,j in zip(var,path):
        globals()[str(i)].append('')
    estadoscrap.append(status)
    #showscreen(driver)

  #Close Driver
  # driver.quit()
  # display.stop_display()
  # time.sleep(2)
  #append data to one law project
  data_tuples = list(zip(titulo,
                        periodo,
                        legisla,
                        fechapresenta,
                        proponente,
                        sumilla,
                        observaciones,
                        autor,
                        coautor,
                        adherentes,
                        grupoparla,
                        comisiones,
                        ultimoestado,
                        estadoscrap,
                        autorlink,
                        fecha,
                        enlace))
  #Convert data tuple into dataframe
  df = pd.DataFrame(data_tuples, columns=vars) # creates dataframe of each tuple in list
    #Append to main data
  dd = pd.concat([dd,df], axis=0, ignore_index=True) # Es la hoja "datoscongreso"
  ddt = pd.concat([ddt,dft], axis=0, ignore_index=True) # Es la hoja "datoscongresoseguimiento"

print(datetime.now() - hora_inicio)

#Close Driver
driver.quit()
display.stop_display()

[WDM] - Downloading: 100%|██████████| 5.53M/5.53M [00:00<00:00, 167MB/s]


2871
ok
3739
ok
3262
ok
3428
not button
ok
4359
ok
4360
ok
4525
ok
4704
ok
4939
ok
0:00:48.925315


In [ ]:
ddt['enlace'].value_counts()

https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/2871    4
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/3739    3
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/3428    3
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/3262    2
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/4359    2
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/4360    2
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/4525    2
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/4704    2
https://wb2server.congreso.gob.pe/spley-portal/#/expediente/2021/4939    2
Name: enlace, dtype: int64

In [ ]:
print('dd', dd.shape)
print('ddt', ddt.shape)

dd (9, 17)
ddt (22, 6)


In [ ]:
# exportación para ver resultados
#lin = 'https://docs.google.com/spreadsheets/d/16vsPetGMBsRQfz-jIYL7JL32euMCYA6O8-qA48L4W8U/edit#gid=0'